## Functions_ABM_Accuracy

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import pandas as pd
from scipy.stats import truncnorm
from scipy.stats import mannwhitneyu
from scipy.stats import wilcoxon
from types import SimpleNamespace
import ipynb.fs.defs.Functions_EXP_Data_Cleaning as dataEXP

In [65]:
def calculateAccuracies(c1):
    # Individual accuracies
    c1 = c1.dropna(axis = 0, subset = ["CHOICE"])
    c1Indv = c1.groupby(by = ["INFLUENCE", "INDIVIDUAL_ID"], as_index = False).mean()
    propIndivIndep = c1Indv[(c1Indv.INFLUENCE == 0)]["CHOICE"]
    propIndivInflu = c1Indv[(c1Indv.INFLUENCE == 1)]["CHOICE"]

    # Group accuracies
    c1Grp = c1.groupby(by = ["INFLUENCE", "GROUP_ID", "QUESTION_ID"], as_index = False).mean()
    c1Grp['GROUP_CORRECT'] = np.where(c1Grp.CHOICE >= 0.5, 1, 0)
    propGroupIndep = c1Grp[c1Grp.INFLUENCE == 0].groupby(by = ['GROUP_ID'])["GROUP_CORRECT"].mean()
    propGroupInflu = c1Grp[c1Grp.INFLUENCE == 1].groupby(by = ['GROUP_ID'])["GROUP_CORRECT"].mean()

    preres = {'propIndivIndep': propIndivIndep, 'propIndivInflu': propIndivInflu, 'propGroupIndep' : propGroupIndep, 'propGroupInflu' : propGroupInflu}
    accs = SimpleNamespace(**preres)
    
    return accs

In [12]:
def printProps(accs):
    print(f"Individual Correctness Proportions\n - Independent: {round(accs.propIndivIndep.mean(),5)}\n - Influenced: {round(accs.propIndivInflu.mean(),5)}") 
    print(f"Group Correctness Proportions\n - Independent: {round(accs.propGroupIndep.mean(),5)}\n - Influenced: {round(accs.propGroupInflu.mean(),5)}")

In [13]:
def printDiffSigs(accs, includeWilcoxon = False):
    U1, p1 = mannwhitneyu(accs.propIndivIndep, accs.propIndivInflu)
    U2, p2 = mannwhitneyu(accs.propGroupIndep, accs.propGroupInflu)
    if includeWilcoxon:
        res1 = wilcoxon(accs.propIndivIndep, accs.propIndivInflu, mode ='approx')
        res2 = wilcoxon(accs.propGroupIndep, accs.propGroupInflu, mode ='approx')
        print(f"Individual accuracy difference: \n  Indep - Influ: {'{:.10f}'.format(accs.propIndivIndep.mean() - accs.propIndivInflu.mean())} \n  Mann-Whitney p-value: {'{:.10f}'.format(p1)}\n  Wilcoxon Signed Rank p-value: {'{:.10f}'.format(res1.pvalue)}")
        print(f"Group accuracy difference: \n  Indep - Influ: {'{:.10f}'.format(accs.propGroupIndep.mean() - accs.propGroupInflu.mean())} \n  Mann-Whitney p-value: {'{:.10f}'.format(p2)}\n  Wilcoxon Signed Rank p-value: {'{:.10f}'.format(res2.pvalue)}")
    else:
        print(f"Individual accuracy difference: \n  Indep - Influ: {'{:.10f}'.format(accs.propIndivIndep.mean() - accs.propIndivInflu.mean())} \n  Mann-Whitney p-value: {'{:.10f}'.format(p1)}")
        print(f"Group accuracy difference: \n  Indep - Influ: {'{:.10f}'.format(accs.propGroupIndep.mean() - accs.propGroupInflu.mean())} \n  Mann-Whitney p-value: {'{:.10f}'.format(p2)}")